# This script defines two funciton to visualize results - a line graph for model training outcomes and a confusion matrix to compared predicted labels with actual labels

In [1]:
import altair as alt
import pandas as pd

In [2]:
train = [.5, .4, .6, .66, .667]
valid = [.9, .45, .56, .6,.7]
test = .6

## Line graph for training, see docstring for function inputs

In [18]:
def training_lines(training_accuracies, validation_accuracies, test_accuracy, title=None):
  '''
  Inputs:
  training_accuracies: list of length n of the training accuracies obtained at each epoch
  validation_accuries: list of length n of the trianing accuracies obtained at each epoch
  test_accuracy: integer of the highest test score attained
  '''
  epochs = list(map(lambda accs: accs+1, range((len(training_accuracies)))))
  train_df= pd.DataFrame({"accuracy":training_accuracies, "epoch":epochs})
  train_df["Dataset"] = "Training accuracy"
  valid_df = pd.DataFrame({"accuracy":validation_accuracies, "epoch":epochs})
  valid_df["Dataset"] = "Validation accuracy"
  test = [test_accuracy]*len(epochs)
  test_df =pd.DataFrame({"epoch":epochs, "accuracy":test})
  test_df['Dataset'] = "Optimum Test accuracy"
  #df = pd.concat([train_df, valid_df, test_df])
  df = pd.concat([train_df, valid_df])
  df['epoch']= df['epoch'].apply(lambda ep: int(ep))
  min_y_axis = min(min(training_accuracies),min(validation_accuracies), test_accuracy) - 0.1

  print(df)

  line_chart = alt.Chart(df).mark_line(point=True).encode(
      x=alt.X("epoch:Q", axis=alt.Axis(tickMinStep=1)),
      y=alt.Y("accuracy", scale=alt.Scale(domain=[min_y_axis, 1])),
      color='Dataset'
  )

  test_line = alt.Chart(test_df).mark_line(strokeDash=[7,1]).encode(
      x=alt.X("epoch:Q", axis=alt.Axis(tickMinStep=1)),
      y=alt.Y("accuracy", scale=alt.Scale(domain=[min_y_axis, 1])),
      color='Dataset'
  )

  return (line_chart + test_line).configure_point(size=15).properties(title={'text': (title if title else "")})

In [19]:
training_lines(train, valid, .5)

   accuracy  epoch              Dataset
0     0.500      1    Training accuracy
1     0.400      2    Training accuracy
2     0.600      3    Training accuracy
3     0.660      4    Training accuracy
4     0.667      5    Training accuracy
0     0.900      1  Validation accuracy
1     0.450      2  Validation accuracy
2     0.560      3  Validation accuracy
3     0.600      4  Validation accuracy
4     0.700      5  Validation accuracy


alt.LayerChart(...)

## Confusion Matrix

This function expects a pandas dataframe with a coulmn for predicted labels and a column for actual labels

In [ ]:
df = pd.DataFrame({"predicted_col":[1,2,3,2,1,1,2,2,3], "actual_col":[1,3,2,2,3,1,3,3,3]})

In [ ]:
df.groupby(["predicted_col", "actual_col"]).size().reset_index(name='count')

,predicted_col,actual_col,count
0,1,1,2
1,1,3,1
2,2,2,1
3,2,3,3
4,3,2,1
5,3,3,1


In [ ]:
 def confusion_matrix(df, predicted_col, actual_col, title=None, share=True):
  matrix = df.groupby([predicted_col, actual_col]).size().reset_index(name='count')
  matrix['share'] = matrix['count'] / len(df)
  max_count = max(matrix['count'])
  base =  alt.Chart(matrix)
  heatmap = base.mark_rect().encode(
      x=alt.X(predicted_col+':O',title="Predicted Class"),
      y=alt.Y(actual_col+':O', title="Actual Class"),
      color=alt.Color(("share" if share else "count")+':Q', scale=alt.Scale(
                        scheme='greens', domain=[0,(1 if share else max_count)]),
                        legend=alt.Legend(
                        title= ("Share" if share else "Count"))
      )
  )
  text = base.mark_text(align='center', baseline='middle').encode(
    x=alt.X(predicted_col+':O'),
    y=alt.Y(actual_col+':O'),
    text='count',
    color=alt.value('black')
    )
  
  return (heatmap + text).properties(
      width=500, height=500,
      title={'text': (title if title else "")})

In [ ]:
confusion_matrix(df, "predicted_col", "actual_col", title="Confusion Matrix", share=False)

alt.LayerChart(...)

In [ ]:
def training_lines(title=None, train_accuracies={}, valid_accuracies={}, test_accuracies={}):
    '''
    Inputs:
    training_accuracies: list of length n of the training accuracies obtained at each epoch
    validation_accuries: list of length n of the trianing accuracies obtained at each epoch
    test_accuracy: integer of the highest test score attained
    '''

    cols = {"value": [], "epoch": [], "metric_name": []}

    # training
    for metric, values in train_accuracies.items():
        n = len(values)
        cols["value"].extend(values)
        cols["epoch"].extend(i+1 for i in range(n))
        cols["metric_name"].extend(["train_" + metric]*n)

    # validation
    for metric, values in valid_accuracies.items():
        n = len(values)
        cols["value"].extend(values)
        cols["epoch"].extend(i+1 for i in range(n))
        cols["metric_name"].extend(["valid_" + metric]*n)
    df = pd.DataFrame(cols)

    # test
    test_cols = {"value": [], "epoch": [], "metric_name": []}
    for metric, value in test_accuracies.items():
        n = len(values)
        test_cols["value"].extend([value] * n)
        test_cols["epoch"].extend(i+1 for i in range(n))
        test_cols["metric_name"].extend(["test_" + metric]*n)

    test_df = pd.DataFrame(test_cols)

    min_y_axis = df['value'].min() - 0.1

    line_chart = alt.Chart(df).mark_line(point=True).encode(
        x=alt.X("epoch:Q", axis=alt.Axis(tickMinStep=1)),
        y=alt.Y("value", scale=alt.Scale(domain=[min_y_axis, 1])),
        color='metric_name')

    test_line = alt.Chart(test_df).mark_line(strokeDash=[7, 1]).encode(
        x=alt.X("epoch:Q", axis=alt.Axis(tickMinStep=1)),
        y=alt.Y("value", scale=alt.Scale(domain=[min_y_axis, 1])),
        color='metric_name')
    
    return (line_chart + test_line).properties(
        title={'text': (title if title else "")})

In [ ]:
train = {"cohens_kappa": [.2, .6, .1, .4, .5],
            "accuracy": [.2, .3, .5, .6, .8],
            "balanced_accuracy": [.1, .4, .2, .1, .8]}
valid = {"cohens_kappa": [.3, .5, .2, .4, .7],
            "accuracy": [.2, .4, .6, .6, .7],
            "balanced_accuracy": [.2, .4, .5, .6, .9]}
test = {"cohens_kappa": .5,
        "accuracy": .6,
        "balanced_accuracy": .7}

chart = training_lines(train_accuracies=train,
                valid_accuracies=valid, test_accuracies=test)

In [ ]:
chart

alt.LayerChart(...)

In [ ]:
def confusion_matrix(predicted, actual, title=""):
    df = pd.DataFrame({"predicted": predicted, "actual": actual})
    df_cells = df.groupby(["predicted", "actual"]
                          ).size().reset_index(name='count')
    df_actual = df.groupby(["actual"]).size().reset_index(name='actual_count')
    df = df_cells.merge(df_actual, how='left', on='actual',
                        left_index=False, right_index=False)
    df['norm'] = df['count'] / df['actual_count']
    # matrix['share'] = matrix['count'] / len(df)
    # max_count = max(matrix['count'])
    base = alt.Chart(df)
    print(df.columns)
    heatmap = base.mark_rect().encode(
        x=alt.X('predicted:O', title="Predicted Class"),
        y=alt.Y('actual:O', title="Actual Class"),
        color=alt.Color('norm:Q', scale=alt.Scale(
                        scheme='greens', domain=[0, 1]),
                        legend=alt.Legend(
                        title="Predicted / Actual")
                        )
    )
    print("made it here")
    text = base.mark_text(align='center', baseline='middle').encode(
        x=alt.X('predicted:O'),
        y=alt.Y('actual:O'),
        text='norm',
        color=alt.value('black')
    )
    print("about to return")
    return (heatmap + text).properties(
        width=500, height=500,
        title={'text': title})

In [ ]:
predicted = [1,2,3,2,1,1,2,2,3,2,3,1,1,2,3,2,1,2,3,2,3,1,2]
actual =    [1,3,2,2,3,1,3,3,3,2,3,3,3,2,1,1,2,3,3,2,3,1,1]
confusion_matrix(predicted, actual, title="Confusion Matrix")

Index(['predicted', 'actual', 'count', 'actual_count', 'norm'], dtype='object')
made it here
about to return


alt.LayerChart(...)

In [ ]:
predicted = [1,2,3,2,1,1,2,2,3]
actual = [1,3,2,2,3,1,3,3,3]
df_cells = df.groupby(["predicted_col", "actual_col"]).size().reset_index(name='count')
df_actual = df.groupby(["actual_col"]).size().reset_index(name='actual_count')
df = df_cells.merge(df_actual, how='left', on='actual_col', left_index=False, right_index=False)
df['norm'] = df['count'] / df['actual_count']
print(df_cells)
print(df_actual)
print(df)baa

   predicted_col  actual_col  count
0              1           1      2
1              1           3      1
2              2           2      1
3              2           3      3
4              3           2      1
5              3           3      1
   actual_col  actual_count
0           1             2
1           2             2
2           3             5
   predicted_col  actual_col  count  actual_count  norm
0              1           1      2             2   1.0
1              1           3      1             5   0.2
2              2           2      1             2   0.5
3              2           3      3             5   0.6
4              3           2      1             2   0.5
5              3           3      1             5   0.2


In [ ]:
confusion_matrix(data, "predicted_col", "actual_col", title="Confusion Matrix", share=False)

alt.LayerChart(...)